In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from xgboost import XGBClassifier # Importamos el clasificador de XGBoost
from sklearn.preprocessing import StandardScaler # Para escalar, si es necesario

In [2]:
# Cargar el archivo de datos
# Asegúrate de que la ruta sea correcta o que el archivo esté en el mismo directorio
try:
    dfC = pd.read_excel("ClusteringClientes.xlsx")
except FileNotFoundError:
    print("Error: Asegúrate de que 'ClusteringClientes.xlsx' esté en el directorio correcto.")
    exit()

In [3]:
dfC.head(2)

,EMAIL,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD,Cluster
0,00-MAZA.ESQUISTOSO@ICLOUD.COM,4,953.664978,3814.659912,6.25,2.5,0.75,190,1,0,0,0,0,0,0,0,1,0,5
1,0022ANDREAGONZALEZ@GMAIL.COM,2,504.309998,1008.619995,0.00,24.5,0.50,781,0,1,0,0,1,0,0,0,0,0,2


In [4]:
# Seleccionar el 90% (tu muestra aleatoria)
df = dfC.sample(frac=0.9)

# Seleccionar el 10% restante (el complemento)
# Se obtienen los índices que NO fueron seleccionados en el paso anterior
df10 = dfC.drop(df.index)

In [5]:
df.head(2)

,EMAIL,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD,Cluster
110731,jose.de.j..a.ramirez.funes@ejemplo.com,1,605.599976,605.599976,0.0,17.0,1.0,35,1,0,0,0,0,1,0,0,0,0,7
1644,ALBERTOGH55@GMAIL.COM,4,726.295044,2905.180176,5.0,15.0,1.0,400,0,1,0,0,0,0,0,0,1,0,3


In [6]:
df10.head(2)

,EMAIL,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD,Cluster
2,01MENDEZ.CESAR@GMAIL.COM,9,1112.066772,10008.600586,5.0,10.0,1.0,183,0,1,0,0,0,0,0,0,1,0,3
3,0209396@UP.ED.MX,2,1034.479980,2068.959961,5.0,4.0,1.0,798,0,1,1,0,0,0,0,0,0,0,2


In [7]:
# 1. Limpieza y Definición de X y y
X = df.drop(['EMAIL', 'Cluster'], axis=1) # Excluir EMAIL y ambas columnas de cluster
y = df['Cluster']
num_clases_final = y.nunique()

In [8]:
X.head(2)

,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD
110731,1,605.599976,605.599976,0.0,17.0,1.0,35,1,0,0,0,0,1,0,0,0,0
1644,4,726.295044,2905.180176,5.0,15.0,1.0,400,0,1,0,0,0,0,0,0,1,0


In [9]:
# 2. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y
)
print(f"Datos listos. Clases finales: {num_clases_final}. Filas de entrenamiento: {X_train.shape[0]}")


# --- 2. Entrenamiento del Modelo XGBoost Básico ---

xgb_model_baseline = XGBClassifier(
    objective='multi:softmax',
    num_class=num_clases_final,
    eval_metric='mlogloss',
    n_estimators=100,
    random_state=42
)

print("\nIniciando entrenamiento básico de XGBoost...")
xgb_model_baseline.fit(X_train, y_train)
y_pred_baseline = xgb_model_baseline.predict(X_test)


# --- 3. Evaluación del Modelo Básico ---

print("\n" + "="*50)
print("--- Evaluación del Modelo Básico ---")

accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Precisión (Accuracy) General: {accuracy:.4f}")

print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred_baseline))


Datos listos. Clases finales: 9. Filas de entrenamiento: 86450

Iniciando entrenamiento básico de XGBoost...

--- Evaluación del Modelo Básico ---
Precisión (Accuracy) General: 0.9989

Informe de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2421
           1       1.00      1.00      1.00      2494
           2       1.00      1.00      1.00      2942
           3       1.00      1.00      1.00      3280
           4       1.00      1.00      1.00       599
           5       1.00      1.00      1.00       883
           6       1.00      1.00      1.00       823
           7       1.00      1.00      1.00     22710
           8       0.98      0.99      0.98       898

    accuracy                           1.00     37050
   macro avg       1.00      1.00      1.00     37050
weighted avg       1.00      1.00      1.00     37050



In [10]:
y_test, y_pred_baseline

(72269     7
 93241     7
 53640     7
 106546    7
 268       0
          ..
 11204     0
 115717    7
 59650     7
 129558    7
 64131     7
 Name: Cluster, Length: 37050, dtype: int64,
 array([7, 7, 7, ..., 7, 7, 7]))

In [11]:
xgb_model_baseline.save_model("modelo_xgboost_clientes.json")

In [12]:
# 1. Limpieza y Definición de X y y
X10 = df10.drop(['EMAIL', 'Cluster'], axis=1) # Excluir EMAIL y ambas columnas de cluster
y10 = df10['Cluster']

In [13]:
X10.head(2)

,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD
2,9,1112.066772,10008.600586,5.0,10.0,1.0,183,0,1,0,0,0,0,0,0,1,0
3,2,1034.479980,2068.959961,5.0,4.0,1.0,798,0,1,1,0,0,0,0,0,0,0


In [15]:
# 1. Crear una nueva instancia (no se necesitan parámetros si se cargará el modelo)
modelo_cargado = XGBClassifier()

# 2. Cargar el modelo guardado
modelo_cargado.load_model("modelo_xgboost_clientes.json")

print("Modelo XGBClassifier cargado exitosamente.")

Modelo XGBClassifier cargado exitosamente.


In [16]:
#y_pred10 = xgb_model_baseline.predict(X10)

# El resultado serán las probabilidades o la puntuación de los clusters (depende de cómo entrenaste).
y_pred10 = modelo_cargado.predict(X10)

print("\nPredicciones:")
print(y_pred10)

accuracy = accuracy_score(y10, y_pred10)
print(f"Precisión (Accuracy) General: {accuracy:.4f}")

print("\nInforme de Clasificación:")
print(classification_report(y10, y_pred10))


Predicciones:
[3 2 2 ... 7 7 7]
Precisión (Accuracy) General: 0.9995

Informe de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       925
           1       1.00      1.00      1.00       912
           2       1.00      1.00      1.00      1164
           3       1.00      1.00      1.00      1199
           4       1.00      1.00      1.00       239
           5       1.00      1.00      1.00       331
           6       1.00      1.00      1.00       301
           7       1.00      1.00      1.00      8299
           8       0.99      0.99      0.99       352

    accuracy                           1.00     13722
   macro avg       1.00      1.00      1.00     13722
weighted avg       1.00      1.00      1.00     13722

